In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from IPython.display import HTML

# let James know if you run into permissions errors
# TODO: replace matplotlib.animation with napari
plt.rcParams['animation.ffmpeg_path'] = '/clusterfs/fiona/jpduncan/.conda/envs/neurohub/bin/ffmpeg'

from os.path import join as oj

import sys
sys.path.append('../src')

from skimage.io import imread, imshow

from puncta import *

### Import data

In [2]:
%%time

data_dir = '/clusterfs/fiona/yugroup/exllsm'
m3_raw = 'm3_Sample4/raw'

#vol_fname = 'Scan1_NC_Iter_0_ch0_CAM1_stack0000_488nm_0000000msec_0117688843msecAbs_000x_000y_000z_0000t.tif'
vol_fname = 'Scan1_NC_Iter_0_ch1_CAM1_stack0000_560nm_0000000msec_0118824472msecAbs_003x_004y_001z_0000t.tif'
im = imread(oj(data_dir, m3_raw, vol_fname))
im.shape

TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes
TiffTag 32781: coercing invalid ASCII to bytes


CPU times: user 23.7 ms, sys: 97.5 ms, total: 121 ms
Wall time: 120 ms


(301, 704, 360)

### Visualize the raw data

In [3]:
fig, ax = plt.subplots()
l = ax.imshow(im[:,0,:], vmin = 0, vmax = np.amax(im))

def animate(i):
    l.set_data(im[:,i,:])

ani = FuncAnimation(fig, animate, frames=im.shape[1])
plt.close()

HTML(ani.to_html5_video())

## Calculate synapse densities

### Parameter defaults (see puncta.py for details)

In [4]:
expansion_factor = 4.09
px_sz = 0.097
px_sz_z = 0.18
sigmas = [2.5, 2.5]
density_vol = 1
find_local_max = True
min_az = 0.1
thresh = None
min_thresh = 0
gauss_kernel_sz = 1
max_quant = .999
min_vox_vol = 516
ball_radius = 2
min_nonzero_frac = 0.1

bw = min_az / 2 / px_sz * expansion_factor # search radius
rad = np.cbrt(density_vol * 3 / 4 / np.pi)
rad_ex = rad * expansion_factor
bw_ex = rad_ex / px_sz # search radius after expansion
struc_el = ball(ball_radius)
px_z_ratio = px_sz_z / px_sz

### Apply Gaussian blur and Otsu thresholding

In [5]:
%%time

if np.sum(im) > 0 and np.count_nonzero(im) >= (im.size * min_nonzero_frac):
    im_gauss = apply_threshold(im, thresh, min_thresh, gauss_kernel_sz, max_quant, verbose=True)
else:
    ValueError(f'There weren\'t enough non-zero voxels in the input volume!')

Otsu's threshold: 351.000
CPU times: user 3.2 s, sys: 1.63 s, total: 4.83 s
Wall time: 4.84 s


### Filter out small artifacts by finding connected components and removing small ones

In [6]:
%%time

if min_vox_vol > 0:
    # filter out small artifacts from the image
    im_gauss = filter_small_artifacts(im_gauss)
    
if np.sum(im_gauss > 0) == 0:
    ValueError(f'There weren\'t any non-zero voxels after thresholding!')

CPU times: user 9min 52s, sys: 2min 39s, total: 12min 32s
Wall time: 12min 33s


### Find the local maximum intensities or puncta centroids if ignoring intensity

In [7]:
%%time

# calculate local maxima or centroids of synapses
if find_local_max:
    all_max = local_max_3d(
        im_gauss, (7, 7, 7),
        clear_border=False
    )
    max_ind = np.nonzero(all_max)
    puncta_ind = np.array([max_ind[0], max_ind[1], max_ind[2]]).T
else:
    centroids = regionprops_table(labels, properties=['centroid'])
    puncta_ind = np.array([centroids['centroid-0'],
                           centroids['centroid-1'],
                           centroids['centroid-2']]).T

if len(puncta_ind[0]) == 1:
    ValueError(f'There was only one synapse centroid/maximum!')

CPU times: user 2.23 s, sys: 605 ms, total: 2.84 s
Wall time: 2.84 s


### Average any local max intensity values that are too close together

In [8]:
%%time

if find_local_max:
    # filter detections closer than 2*bw, replacing with the mean of close points
    puncta_ind = remove_duplicates(puncta_ind, 2*bw)

CPU times: user 5.16 ms, sys: 45 µs, total: 5.21 ms
Wall time: 4.83 ms


### Calculate local neighborhood synapse densities

In [9]:
%%time

# stretch out the z axis
puncta_ind_z = puncta_ind.copy()
puncta_ind_z[:, 2] = puncta_ind_z[:, 2] * px_z_ratio

# get the number of puncta_ind within a radius of bw_ex of each point and
# divide by density_vol to calculate the synapse density
tree = KDTree(puncta_ind_z)
neighbor_count = tree.query_radius(puncta_ind_z, r=bw_ex, count_only=True)
synapse_density = neighbor_count / density_vol

mask = np.zeros(im.shape)
mask[puncta_ind[:, 0], puncta_ind[:, 1], puncta_ind[:, 2]] = synapse_density
mask = dilation(mask, struc_el)

nonzero_idx = im_gauss != 0
clean_vol = np.zeros(im.shape)
clean_vol[nonzero_idx] = im[nonzero_idx]

CPU times: user 2.83 s, sys: 416 ms, total: 3.25 s
Wall time: 3.25 s


## Visualize results

### Plot the puncta mask

In [10]:
fig, ax = plt.subplots()
l = ax.imshow(mask[:,0,:], vmin = 0, vmax = np.amax(mask))

def animate(i):
    l.set_data(mask[:,i,:])

ani = FuncAnimation(fig, animate, frames=mask.shape[1])
plt.close()

from IPython.display import HTML
HTML(ani.to_html5_video())

### Plot the cleaned volume data

In [11]:
fig, ax = plt.subplots()
l = ax.imshow(clean_vol[:,0,:], vmin = 0, vmax = np.amax(clean_vol))

def animate(i):
    l.set_data(clean_vol[:,i,:])

ani = FuncAnimation(fig, animate, frames=clean_vol.shape[1])
plt.close()

from IPython.display import HTML
HTML(ani.to_html5_video())

### Plot the raw data with puncta mask super-imposed

In [13]:
im_cp = im.copy()
im_cp[mask != 0] = np.amax(im)

fig, ax = plt.subplots()
l = ax.imshow(im_cp[:,0,:], vmin = 0, vmax = np.amax(im_cp))

def animate(i):
    l.set_data(im_cp[:,i,:])

ani = FuncAnimation(fig, animate, frames=im_cp.shape[1])
plt.close()

HTML(ani.to_html5_video())